# Analysis of Dopaminergic Terminal Detector (DTD) model

## 0. Outline
This code deals with the automatic processing of raw data from mouse brains analysed with “Dopaminergic Terminal Detector” model developed in Aiforia® Create. We typically start from Excel/CSV files collected in a local folder on the computer that is specified in the code. To automatically change the format of a series of files, refer to the to the Change_Name_Format_Input_Data.ipynb notebook. The code is developed to take into account that a mouse brain can be mounted over several slides. Slides for each animal are named identically, except for a numeric postfix denoting the slide number: '_S1', '_S2', etc. For this code it is required that mouse brain sections containing the striatum from the same mouse are all mounted on the same '_S1' slide.

The present notebook is divided into 3 sections:

**1) Make the necessary functions for part 2**

**2) Automatic Analysis of N Slides (of which the name contains '_S1') of N Brains**

Here we automate the analysis of all N S1 slides of all N brains (1 slide per brain) in the folder with raw data. The approach is as follows:

1) We collect all the N names of the S1 raw data files in the folder and store them in a list.

2) We loop over these N slides belonging to the N brains and perform the following steps in each loop:

    a) We perform the data analysis steps.  
    b) We output the results to an excel file for this specific brain.
    
After each loop, we add the output of this specific brain to an overview table that will contain all results for all brains. After the last loop, this overview table is also exported to an excel file.

In this part of the code, we **do not** make more detailed **'transposed'** tables with information on each Striatum part. 

**3) Automatic Analysis of N Slides (of which the name contains '_S1') of N Brains after determining to which hemisphere they belong**

Here we add which Striatum regions are on each hemisphere, and compare the injected vs non-injected sides. Analysis occurs similar to section 2. In this part of the code, we also make more detailed **'transposed'** tables with information on each Striatum part for easier visualization.

## Part 1 - Make the necessary functions


### Part 1.1 - Load all necessary Python packages

In [ ]:
# Import the required Python packages
import pandas as pd                                # For data analysis with dataframes
import math                                        # To get the value for pi
import functools                                   # For higher-order functions that work on other functions
from IPython.display import display                # Enables the display of more than one dataframe per code cell
import numpy as np                                 # For data analysis
import glob                                        # To get all raw data file locations
import os                                          # To get all raw data file locations
pd.options.display.float_format = '{:.2f}'.format  # Display all numbers in dataframes with 2 decimals
import re                                          # To do Regular Expressions

### Part 1.2 - Data locations

**TO DO:** 
- Specify the format of the raw data and the raw data folder location, as well as some experimental parameters.
- Specify the file paths of the excel file containing your quality control revisions and the excel file mapping each brain region to a hemisphere.
- Specify the folder locations where you would like to collect the output excel files (for whole brain and hemisphere analysis).  

The format is: <font color='darkred'>r'file_location'</font> 

In [ ]:
# Specify what data format you want to use for your raw data: excel, csv or feather. Do this by uncommenting the data_format that you want.
# data_format = 'excel'
data_format = 'csv'
# data_format = 'feather'

# Specify the experimental parameters (section_thickness in micrometers) and locations:
# The spacing parameter refers to the serial section spacing interval. It's the interval at which you sample the brain volume for analysis, not the physical distance between each section. For example, if you have a spacing parameter of 10, you would take every 10th section for your analysis.  
spacing=12
section_thickness = 40
folder_raw_data = r'C:\Users\...\Raw_Data_DTD'
file_brainregions_to_replace =  r'C:\Users\...\Brainregions_To_Replace_DTD.xlsx'
file_brainregions_injected =  r'C:\Users\...\Brainregions_Hemisphere_DTD.xlsx'
folder_output_results = r'C:\Users\...\Results_Wholebrain_DTD'
folder_output_results_injected = r'C:\Users\...\Results_Hemisphere_DTD'


In [ ]:
# Make the output folders if they did not exist yet
if not os.path.isdir(folder_output_results):
    os.mkdir(folder_output_results)
if not os.path.isdir(folder_output_results_injected):
    os.mkdir(folder_output_results_injected)

### Part 1.3 - Function to load all image files that need to be analyzed

In [ ]:
def load_all_file_locations_S1(folder_raw_data):
    """
    Make a list of all file locations for S1 images present in the folder with all raw data files. There are only supposed to be S1 images in the folder, 
    so '_S1' does not have to be in the filename here.
    Output: list of all file locations for S1 images.
    """

    if data_format == 'excel':
        all_raw_data_file_locations_S1 = glob.glob(os.path.join(folder_raw_data, "*.xlsx"))
    elif data_format == 'csv':
        all_raw_data_file_locations_S1 = glob.glob(os.path.join(folder_raw_data, "*.csv"))
    elif data_format == 'feather':
        all_raw_data_file_locations_S1 = glob.glob(os.path.join(folder_raw_data, "*.feather"))
    else:
        print('You did not specify a correct data-format in Part 1.2 and can expect some errors in the rest of the code')
        
    all_raw_data_file_locations_S1.sort()

    print('The location of all the raw data files = ')
    for file_location_S1 in all_raw_data_file_locations_S1:
        print(file_location_S1)

      
    return all_raw_data_file_locations_S1

### Part 1.4 - Function to load the file with corrections for the brainregions

In [ ]:
def load_data_brainregions_to_replace(file_brainregions_to_replace):
    """
    Load the file containing the corrections for brain regions that need to be replaced for each specific image.
    Output: cleaned dataframe with brain regions that need to be replaced for each image.
    """
    
    df_brainregions_to_replace_raw=pd.read_excel(file_brainregions_to_replace,
                                                 usecols=['Image', 'Brainregion_Wrong', 'Brainregion_Correct'],
                                                 dtype={'Image': 'str', 'Brainregion_Wrong': 'str', 'Brainregion_Correct': 'str'}
                                                )

    # Modify the dataframe to delete spaces that are by accident there, and put the brainregions in upper case 
    df_brainregions_to_replace=df_brainregions_to_replace_raw.copy()
    df_brainregions_to_replace['Image'] = df_brainregions_to_replace_raw['Image'].str.strip()
    df_brainregions_to_replace['Brainregion_Wrong'] = df_brainregions_to_replace_raw['Brainregion_Wrong'].str.upper().str.strip()
    df_brainregions_to_replace['Brainregion_Correct'] = df_brainregions_to_replace_raw['Brainregion_Correct'].str.upper().str.strip()

    #     print('The raw table of the brain regions to replace for each image = ')
    #     display(df_brainregions_to_replace_raw)

    print('The modified table of the brain regions to replace for each image = ')
    display(df_brainregions_to_replace)
    
    return df_brainregions_to_replace

### Part 1.5 - Function to load the file with which brainregions were injected


In [ ]:
def load_data_brainregions_injected(file_brainregions_injected):
    """
    Load the file specifying which brainregions were on the injected side for each specific image.
    Output: cleaned dataframe with brain regions that were injected for each image.
    """
    
    df_brainregions_injected_raw=pd.read_excel(file_brainregions_injected,
                                               usecols=['Image', 'Brainregion', 'Hemisphere'],
                                               dtype={'Image': 'str', 'Brainregion': 'str', 'Hemisphere': 'str'}
                                               )

    # Modify the dataframe to delete spaces that are by accident there, and put the brainregions in upper case 
    df_brainregions_injected=df_brainregions_injected_raw.copy()
    df_brainregions_injected['Image'] = df_brainregions_injected_raw['Image'].str.strip()
    df_brainregions_injected['Brainregion'] = df_brainregions_injected_raw['Brainregion'].str.upper().str.strip()
    df_brainregions_injected['Region_Hemisphere'] = df_brainregions_injected_raw['Hemisphere'].str.upper().str.strip()
    df_brainregions_injected.drop(columns=['Hemisphere'], inplace=True)

    #     print('The raw table of the brain regions injected for each image = ')
    #     display(df_brainregions_injected_raw)

    print('The modified table of the brain regions injected for each image = ')
    display(df_brainregions_injected)
    
    return df_brainregions_injected

### Part 1.6 - Function to load dataframe and clean it


In [ ]:
def dataframe_cleaning(file_location, df_brainregions_to_replace):
    """
    Load the specific file location in a dataframe and clean it with df_brainregions_to_replace.
    Output: loaded and cleaned dataframe with some additional calculated values.
    """
    
    # Load file with raw data for image S1 of this specific brain
    if data_format == 'excel':
        df_1=pd.read_excel(file_location,
                           usecols=['Image', 'Area/object name', 'Area (μm²)', 'B', 'G', 'R'],
                           dtype={'Image': 'str', 'Area/object name': 'str', 'Area (μm²)': 'float64', 
                                  'B': 'float64', 'G': 'float64', 'R': 'float64' },
                           keep_default_na = True)
    elif data_format == 'csv':
        df_1=pd.read_csv(file_location, sep='\t',
                         usecols=['Image', 'Area/object name', 'Area (μm²)', 'B', 'G', 'R'],
                           dtype={'Image': 'str', 'Area/object name': 'str', 'Area (μm²)': 'float64', 
                                  'B': 'float64', 'G': 'float64', 'R': 'float64' },
                         keep_default_na = True)
    elif data_format == 'feather':
        df_1=pd.read_feather(file_location) 
        dtype_dictionary = {'Image': 'object', 'Area/object name': 'object', 'Area (μm²)': 'float64', 
                            'B': 'float64', 'G': 'float64', 'R': 'float64' }
        df_1=df_1.astype(dtype_dictionary)
    else:
        print('You did not specify a correct data format in Part 1.2 and can expect some errors in the rest of the code')
   
    
        
    # Get the image name out of the file_path (getting image name from dataframe first column is hard because some are empty, 
    # and getting from filename makes more sense anyway) 
    full_name = os.path.basename(file_location)
    file_name = os.path.splitext(full_name)
    image_name = file_name[0]
    print('The present image=', image_name)

    # Make sure the image name across the whole first column is correct
    df_1['Image']=image_name
    
    # Delete the rows with an empty  Area (μm²) or Area/object name 
    # df_1.dropna(subset =['Area (μm²)', 'Area/object name'] , how='any', inplace=True)
    
    # Put all columns in capitals to never make mistakes against capitalization
    df_1['Area/object name'] = df_1['Area/object name'].str.upper()

    # Brain tissue detector rows are not needed
    df_2 = df_1[ ~(df_1['Area/object name'].str.contains("BRAIN TISSUE DETECTOR")) ]

    # Determine the dictionary of brain regions that should be replaced for this specific image
    df_brainregions_to_replace = df_brainregions_to_replace[df_brainregions_to_replace['Image']==image_name]
    dict_brainregions_to_replace= pd.Series(df_brainregions_to_replace.Brainregion_Correct.values, index=df_brainregions_to_replace.Brainregion_Wrong).to_dict()

    print('The dictionary of brain regions to replace for this specific image', image_name, 'is', dict_brainregions_to_replace)

    # Replace the value in the rows that have an Area/object name that is in list_brainregions_replace
    df_3 = df_2.copy()
    df_3['Area/object name'] = df_3['Area/object name'].replace(dict_brainregions_to_replace, regex=False)

    # Create a column 'Area/object name merged' where the numbers are deleted from these columns:
    df_3['Area/object name merged'] = df_3['Area/object name'].str.replace('\\d+', '', regex=True).str.strip()

    # Convert RGB values to grey scale intensity
    df_4 = df_3.copy() 
    # The higher the 'lightness' value, the lighter the image as white is RGB=(255,255,255) and black = RGB(0,0,0). We use a scale 0 to 100, but boils down to the same
    df_4['Lightness'] = 0.299*df_3['R'] + 0.587*df_3['G'] + 0.114*df_3['B'] 
    # df_4['Lightness'] = 1/3*df_3['R'] + 1/3*df_3['G'] + 1/3*df_3['B'] 
    # The higher the 'intensity' value, the darker / more intense the image.
    df_4['Intensity'] = 100 - df_4['Lightness']

    print('The full data=')
    display(df_4)
 
    return df_4


### Part 1.7 - Function to calculate all information (disregarding injected/uninjected)


In [ ]:
def all_calculations(df1, groupby_column1='Area/object name merged'):
    """
    Make the main calculations of total area and weighted intensities for each Area/object name merged
    Output: dataframe with all calculations for each Area/object name merged.
    """

    # Count the number of rows for each Area/object name merged
    df_counts = df1.value_counts(groupby_column1, sort=True).reset_index(name='Counts')

    # Calculate the total area of each Area/object name merged
    df_total_area = df1.groupby(groupby_column1).sum(numeric_only=True)['Area (μm²)'].rename_axis(groupby_column1).reset_index(name='Total Region Area (μm²)')

    # Calculate the weighted intensity of each Area/object name merged
    f = lambda x: sum(x['Intensity'] * x['Area (μm²)']) / sum(x['Area (μm²)'])
    df_weighted_intensity= df1.groupby(groupby_column1).apply(f).rename_axis(groupby_column1).reset_index(name='Weighted Intensity')
    
    # Put all calculated results together
    dfs_to_merge = [df_counts, df_total_area, df_weighted_intensity]
    df_all_calcs  = functools.reduce(lambda left, right: pd.merge(left,right,on=groupby_column1, how='outer'), dfs_to_merge)

    df_all_calcs['Total Region Area (mm²)'] = df_all_calcs['Total Region Area (μm²)'] / 1000000
    df_all_calcs['Total Region Volume (mm³)'] =  df_all_calcs['Total Region Area (mm²)']*(section_thickness/1000)*spacing
    
    df_all_calcs.drop(columns=['Total Region Area (μm²)'], inplace=True)
    df_all_calcs.sort_values(by=[groupby_column1], ascending=False, inplace=True)
    
    print('Calculations for each Area/object name merged')
    display(df_all_calcs)

    return df_all_calcs


### Part 1.8 - Function to calculate all information for the injected/uninjected hemispheres


In [ ]:
def all_calculations_injected(df_injected_object, groupby_column1='Region_Hemisphere'):
    """
    Make the main calculations of total area and weighted intensities for each hemisphere region 
    Output: dataframe with all calculations with the specification injected/uninjected side.
    """

    # Count the number of rows for each Region_Hemisphere
    df_counts_injected = df_injected_object.value_counts(groupby_column1, sort=True).reset_index(name='Counts')

    # Calculate the total area of each Region_Hemisphere
    df_total_area_injected = df_injected_object.groupby(groupby_column1).sum(numeric_only=True)['Area (μm²)'].rename_axis(groupby_column1).reset_index(name='Total Region Area (μm²)')

    # Calculate the weighted intensity of each Region_Hemisphere
    f = lambda x: sum(x['Intensity'] * x['Area (μm²)']) / sum(x['Area (μm²)'])
    df_weighted_intensity_injected= df_injected_object.groupby(groupby_column1).apply(f).rename_axis(groupby_column1).reset_index(name='Weighted Intensity')
    
    # Put all calculated results together
    dfs_to_merge = [df_counts_injected, df_total_area_injected, df_weighted_intensity_injected]
    df_all_calcs_injected  = functools.reduce(lambda left, right: pd.merge(left,right,on=groupby_column1, how='outer'), dfs_to_merge)

    df_all_calcs_injected['Total Region Area (mm²)'] = df_all_calcs_injected['Total Region Area (μm²)'] / 1000000
    df_all_calcs_injected['Total Region Volume (mm³)'] =  df_all_calcs_injected['Total Region Area (mm²)']*(section_thickness/1000)*spacing
    
    df_all_calcs_injected.sort_values(by=[groupby_column1], ascending=False, inplace=True)
    
    print('Calculations on injected/uninjected regions')
    display(df_all_calcs_injected)

    return df_all_calcs_injected

### Part 1.9 - Function to display all information of the different Striatums and hemispheres (transposed)


In [ ]:
def all_calculations_transposed(df1, df2, filter_column1='Region_Hemisphere'):
    """
    Output:  the area and intensity for each brain section (Striatum X), as well as the total area and weighted intensity 
    for the two hemispheres (injected/uninjected) that we calculated in section 1.7. 
    """

    df_short = df1[['Region_Hemisphere', 'Area/object name', 'Intensity', 'Area (μm²)']]
    df_short2= df2[['Region_Hemisphere', 'Weighted Intensity', 'Total Region Area (μm²)']]
    image_name=df1['Image'][0]

    dictionary_inj_uninj={}
    for region in ['STRIATUM UNINJECTED', 'STRIATUM INJECTED']:
        inj_uninj=region.split()[-1]   # This is either UNINJECTED or INJECTED
        
        df_short_transp = df_short[df_short[filter_column1] == region][['Area/object name', 'Intensity', 'Area (μm²)']].transpose()
        
        # Change columns to UNINJECTED1', 'UNINJECTED2' ...  or 'INJECTED1', 'INJECTED2' ... 
        df_short_transp.columns = [inj_uninj+ str(x) for x in range(1, len(df_short_transp.columns)+1)]    
        # Rename index
        df_short_transp = df_short_transp.rename(index={'Area/object name': image_name + ' Area name',
                                                        'Intensity': image_name + ' Intensity',
                                                        'Area (μm²)': image_name + ' Area (μm²)' })
        # display(df_short_transp)
        
        df_short2_transp = df_short2[df_short2[filter_column1]==region].transpose()
        # Change columns to 'TOTAL UNINJECTED'   or 'TOTAL INJECTED'
        df_short2_transp.columns = ['TOTAL ' + inj_uninj]
        # Rename index
        df_short2_transp = df_short2_transp.rename(index={'Region_Hemisphere': image_name + ' Area name',
                                                          'Weighted Intensity': image_name + ' Intensity',
                                                          'Total Region Area (μm²)': image_name + ' Area (μm²)' })
        # display(df_short2_transp)
    
        # Concatenate the dataframes together horizontally:
        df_concatenated = pd.concat([df_short_transp,df_short2_transp], axis=1)
        
        print(f'Transposed {region} regions with all info')
        display(df_concatenated)
        dictionary_inj_uninj[region]=df_concatenated
    
    # Concatenate the dataframe with the uninjected info and the injected info together horizontally:
    df_uninjected_injected_info = pd.concat(dictionary_inj_uninj.values(), axis=1)

    a = pd.to_numeric(df_uninjected_injected_info['TOTAL INJECTED'], errors='coerce')
    b = pd.to_numeric(df_uninjected_injected_info['TOTAL UNINJECTED'], errors='coerce')
    df_uninjected_injected_info['Loss'] = 100*(1-a/b)

    # Bring the Loss column to the front
    df_uninjected_injected_info = df_uninjected_injected_info[ ['Loss'] + [ col for col in df_uninjected_injected_info.columns if col != 'Loss' ] ]
    
    print('Dataframe with the uninjected info and the injected info together')
    display(df_uninjected_injected_info)
    
    return df_uninjected_injected_info

## Part 2 - Automatic Wholebrain Analysis of all N S1 Slides of all N Brains 


In [ ]:
%%time   
# For curiosity we measure the time the code in this cell takes to run

# Load the modified file with brain regions to replace/delete for each specific image 
df_brainregions_to_replace=load_data_brainregions_to_replace(file_brainregions_to_replace)

# Extract the file names that contain '_S1' in the file name. These are the N first images of the N unique brains.
all_raw_data_file_locations_S1= load_all_file_locations_S1(folder_raw_data)

# We initiate a counter to keep track in which loop we are below:
count = 0

# Loop over all the S1 pictures in the raw_data folder
for file_location_S1 in all_raw_data_file_locations_S1:
    count = count +1 # Counts the loop; first loop: counter = 1

    # Get the image name out of the file_path 
    full_name = os.path.basename(file_location_S1)
    file_name = os.path.splitext(full_name)
    image_name_S1 = file_name[0]
    
    # Do the S1 data cleaning, making use of the functions defined above
    print('\n Analysis of ', file_location_S1)
    df_S1_final = dataframe_cleaning(file_location_S1, df_brainregions_to_replace)

    # Do all the calculations, making use of the functions defined above. 
    df_S1_all_calcs = all_calculations(df_S1_final)

    
    ############################
    #   Prepare overview file  #
    ############################
    

    # For the overview excel file, only the df_S1_all_calcs dataframe is needed. 
    # We will make 1 overview excelfiles with a few tabpages that we store in dictionary_overview_dataframes:
    # dictionary_overview_dataframes = {Total Intensity : df,  Total Region Area: df, .... }
    
    # In the first loop we initiate an empty overview dictionary that will be filled with dataframes. 
    if count==1:
        dictionary_overview_dataframes={}
     
    # Prepare the dataframes that are needed for the overview excel file: choose the needed columns,
    # and rename the header of the column with the values to the image_name 
    list_calculation_results=['Weighted Intensity', 'Total Region Area (mm²)', 'Total Region Volume (mm³)']

    for calculation_result in list_calculation_results:
        df_S1_all_calcs_calculation= df_S1_all_calcs[['Area/object name merged', calculation_result]].copy()
        df_S1_all_calcs_calculation.rename(columns={calculation_result: image_name_S1}, inplace=True)
        
        if count==1:
            # In the first loop we fill the empty overview dictionary with a dataframe with the values calculated in loop 1  
            dictionary_overview_dataframes[calculation_result]  = df_S1_all_calcs_calculation.copy()
            
        elif count > 1 :
            # In the subsequent loops we will add the values of those loops to the dataframes in the overview dictionary
            dictionary_overview_dataframes[calculation_result] = dictionary_overview_dataframes[calculation_result].merge(df_S1_all_calcs_calculation, how='outer', on='Area/object name merged')

    # At the end, we delete some of the dataframes, to ensure they cannot be used in the next loop
    del(df_S1_final)
    del(df_S1_all_calcs)
                
# After the for loops, we print the final overview tables

# Output the final overview tables to an excel file Overview_THTerminals_Results.xlsx that is created in the output folder specified at the beginning of this notebook
output_file_name_overview = os.path.join(folder_output_results, 'Overview_THTerminals_Results.xlsx')

list_calculation_results=['Weighted Intensity', 'Total Region Area (mm²)', 'Total Region Volume (mm³)']
    
with pd.ExcelWriter(output_file_name_overview) as writer: 
    for calculation_result in list_calculation_results:
        calculation_result_clean = calculation_result.replace('/', ' per ')
        
        print(f'Overview dataframe with all {calculation_result_clean} for all brains')
        display(dictionary_overview_dataframes[calculation_result])
        dictionary_overview_dataframes[calculation_result].to_excel(writer, sheet_name=calculation_result_clean, index=False, float_format = "%.3f")


## Part 3 - Automatic Hemisphere Analysis of all N S1 Slides of all N Brains (injected vs uninjected)


In [ ]:
%%time   
# For curiosity we measure the time the code in this cell takes to run

# Load the modified file with brain regions to replace/delete for each specific image 
df_brainregions_to_replace=load_data_brainregions_to_replace(file_brainregions_to_replace)

# Load the modified file with hemisphere analysis for each specific image 
df_brainregions_injected=load_data_brainregions_injected(file_brainregions_injected)

# Extract the file names that contain '_S1' in the file name. These are the N first images of the N unique brains.
all_raw_data_file_locations_S1= load_all_file_locations_S1(folder_raw_data)

# We initiate a counter to keep track in which loop we are below:
count = 0

# Loop over all the S1 pictures in the raw_data folder
for file_location_S1 in all_raw_data_file_locations_S1:
    count = count +1 # Counts the loop; first loop: counter = 1

    # Get the image name out of the file_path 
    full_name = os.path.basename(file_location_S1)
    file_name = os.path.splitext(full_name)
    image_name_S1 = file_name[0]
    
    # Do the S1 data cleaning, making use of the functions defined above
    print('\n Analysis of ', file_location_S1)
    df_S1_final = dataframe_cleaning(file_location_S1, df_brainregions_to_replace)

    # Inner join with the injected/uninjected areas that are in the Brainregions_Hemisphere_DTD.xlsx file. 
    df_injected_object= df_brainregions_injected.merge(df_S1_final, left_on=['Image', 'Brainregion'], right_on=['Image', 'Area/object name'], how='inner')
    df_injected_object.drop(columns=['Brainregion'], inplace=True)

    # Do all the calculations, making use of the functions defined above. 
    df_S1_all_calcs_injected = all_calculations_injected(df_injected_object)
    df_uninjected_injected_info = all_calculations_transposed(df_injected_object, df_S1_all_calcs_injected)

    
    ############################
    #   Prepare overview file  #
    ############################
    

    # For the overview excel file, only the df_S1_all_calcs_injected and df_uninjected_injected_info dataframe is needed. 
    # We will make 1 overview excelfiles with a few tabpages that we store in dictionary_overview_dataframes:
    # dictionary_overview_dataframes = {Intensities and Areas : df,  .... }
    
    # In the first loop we initiate an empty overview dictionary that will be filled with dataframes. 
    if count==1:
        dictionary_overview_dataframes={}
        dictionary_overview_dataframes['Intensities and Areas']= df_uninjected_injected_info
    elif count > 1 : 
        dictionary_overview_dataframes['Intensities and Areas']  = pd.concat([dictionary_overview_dataframes['Intensities and Areas'], df_uninjected_injected_info])   

     
    # Prepare the dataframes that are needed for the overview excel file: choose the needed columns,
    # and rename the header of the column with the values to the image_name 
    list_calculation_results=['Total Region Area (mm²)', 'Total Region Volume (mm³)']

    for calculation_result in list_calculation_results:
        df_S1_all_calcs_injected_calculation= df_S1_all_calcs_injected[['Region_Hemisphere', calculation_result]].copy()
        df_S1_all_calcs_injected_calculation.rename(columns={calculation_result: image_name_S1}, inplace=True)
        
        if count==1:
            # In the first loop we fill the empty overview dictionary with a dataframe with the values calculated in loop 1  
            dictionary_overview_dataframes[calculation_result]  = df_S1_all_calcs_injected_calculation.copy()
            
        elif count > 1 :
            # In the subsequent loops we will add the values of those loops to the dataframes in the overview dictionary
            dictionary_overview_dataframes[calculation_result] = dictionary_overview_dataframes[calculation_result].merge(df_S1_all_calcs_injected_calculation, how='outer', on='Region_Hemisphere')

    # At the end, we delete some of the dataframes, to ensure they cannot be used in the next loop
    del(df_S1_final)
    del(df_injected_object)
    del(df_S1_all_calcs_injected)
    del(df_uninjected_injected_info)
                
# After the for loops, we print the final overview tables

# Start by rearranging the columns of the dataframe dictionary_overview_dataframes['Intensities and Areas']: 
# first the Loss %, then the Uninjected columns, then the Injected columns
columns_to_order = dictionary_overview_dataframes['Intensities and Areas'].columns
columns_0 = ['Loss']
columns_1 = [x for x in columns_to_order if 'UNINJECTED' in x and 'TOTAL' not in x]
columns_1.sort(key= lambda x: float(x[10:]))  # to sort based on the number in it, and make sure that the order is U1, U2 ... U9, U10... instead of U1, U10, U2...
columns_2 = ['TOTAL UNINJECTED']
columns_3 = [x for x in columns_to_order if 'INJECTED' in x and 'UN' not in x and 'TOTAL' not in x]
columns_3.sort(key= lambda x: float(x[8:])) 
columns_4 = ['TOTAL INJECTED']
columns_ordered = columns_0 + columns_1 + columns_2 + columns_3 + columns_4
print(columns_ordered)
# Now reorder the columns of the dataframe:
dictionary_overview_dataframes['Intensities and Areas'] = dictionary_overview_dataframes['Intensities and Areas'][columns_ordered]

# Output the final overview tables to an excel file Overview_THTerminals_Hemisphere_Results.xlsx that is created in the output folder specified at the beginning of this notebook
output_file_name_overview = os.path.join(folder_output_results_injected, 'Overview_THTerminals_Hemisphere_Results.xlsx')

df=dictionary_overview_dataframes['Intensities and Areas']
df_loss_intensities=df[df.index.str.contains('Intensity')]
dictionary_overview_dataframes['Intensity Loss'] = df_loss_intensities['Loss']


list_calculation_results=['Intensity Loss', 'Intensities and Areas', 'Total Region Area (mm²)', 'Total Region Volume (mm³)']
    
with pd.ExcelWriter(output_file_name_overview) as writer: 
    for calculation_result in list_calculation_results:
        calculation_result_clean = calculation_result.replace('/', ' per ')
        
        print(f'Overview dataframe with all {calculation_result_clean} for all brains')
        display(dictionary_overview_dataframes[calculation_result])
        if calculation_result in ('Intensity Loss', 'Intensities and Areas'):
            dictionary_overview_dataframes[calculation_result].to_excel(writer, sheet_name=calculation_result_clean, index=True, float_format = "%.3f")
        else:
            dictionary_overview_dataframes[calculation_result].to_excel(writer, sheet_name=calculation_result_clean, index=False, float_format = "%.3f")
